In [8]:
import pandas as pd
import numpy as np
import requests
import html5lib
from bs4 import BeautifulSoup
from datetime import datetime
from openpyxl import workbook
import re
import os
import webbrowser
import unicodedata
from Helper_package import Helper

In [9]:
headers = {
    'User-Agent': 'Goldman Sachs BDC, Inc. GSBD on NYSE'
}
CIK = '0001572694'

filing_data = Helper.fetch_filing_data(cik=CIK, headers=headers)
if filing_data is not None:
    # Write DataFrame to Excel file with auto-adjusting column widths
    file_name = "../GSBC_sec_filing_links.xlsx"
    with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
        filing_data.to_excel(writer, index=False)
        worksheet = writer.sheets['Sheet1']
        for i, col in enumerate(filing_data.columns):
            column_len = max(filing_data[col].astype(
                str).str.len().max(), len(col)) + 2
            worksheet.set_column(i, i, column_len)

    print(f"Data written to {file_name}")

filing_links = Helper.get_filing_links('../GSBC_sec_filing_links.xlsx')
print("Filing link recived")

Data written to ../GSBC_sec_filing_links.xlsx
Filing link recived


In [10]:
filing_links['reportDate'] = pd.to_datetime(
    filing_links['reportDate']).dt.strftime("%B %d, %Y")
print("Date was converted to '%B %d, %Y' format and back to")
filing_links['reportDate'].info()

Date was converted to '%B %d, %Y' format and back to
<class 'pandas.core.series.Series'>
RangeIndex: 36 entries, 0 to 35
Series name: reportDate
Non-Null Count  Dtype 
--------------  ----- 
36 non-null     object
dtypes: object(1)
memory usage: 416.0+ bytes


In [11]:
filing_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   accessionNumber        36 non-null     object 
 1   filingDate             36 non-null     object 
 2   reportDate             36 non-null     object 
 3   acceptanceDateTime     36 non-null     object 
 4   act                    36 non-null     int64  
 5   form                   36 non-null     object 
 6   fileNumber             36 non-null     object 
 7   filmNumber             36 non-null     int64  
 8   items                  0 non-null      float64
 9   size                   36 non-null     int64  
 10  isXBRL                 36 non-null     int64  
 11  isInlineXBRL           36 non-null     int64  
 12  primaryDocument        36 non-null     object 
 13  primaryDocDescription  36 non-null     object 
 14  fileLink               36 non-null     object 
 15  txtFileL

In [12]:
def check_links_validity(filing_links):
    '''
        Checks the validity of each file link in the DataFrame.
    '''
    valid_links = []
    invalid_links = []
    for index, row in filing_links.iterrows():
        link = row['fileLink']
        try:
            response = requests.head(link, headers=headers)
            response.raise_for_status()
            valid_links.append(link)
        except Exception as e:
            invalid_links.append((link, str(e)))
    if invalid_links:
        print("\nInvalid Links:")
        for link, error_message in invalid_links:
            print(f"{link}: {error_message}")

    if len(valid_links) == filing_links.shape[0]:
        print("All Valid Links")


check_links_validity(filing_links)

All Valid Links


In [13]:
test_file = filing_links.iloc[0]
print(test_file)
content = Helper.get_content(Helper.get_response(
    url=test_file['fileLink'], headers=headers))

accessionNumber                                       0000950170-24-022198
filingDate                                                      2024-02-28
reportDate                                               December 31, 2023
acceptanceDateTime                                2024-02-28T16:51:23.000Z
act                                                                     34
form                                                                  10-K
fileNumber                                                       814-00998
filmNumber                                                        24696411
items                                                                  NaN
size                                                              43314172
isXBRL                                                                   1
isInlineXBRL                                                             1
primaryDocument                                          gsbd-20231231.htm
primaryDocDescription    

In [14]:
def extract_tables(content, report_Date) -> pd.DataFrame:
    master_table = pd.DataFrame()
    print(f"Extractiong File: {report_Date}")
    consolidated_schedule_regex = re.compile(r'(?i)^\s*.*\s*SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    




extract_tables(content=content, report_Date=test_file['reportDate'])